# UC7 Network Connectivity

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK


This notebook is meant as supplementary material to the more expansive [UC7 E2E notebook](./UC7-E2E-Demo.ipynb).

Focusing on the simulation components of UC7, this notebook primarily intends to demonstrate the default connectivity and prove that the simulation is operating as expected. Additionally, this notebook assumes that the reader is familiar with both UC7 and PrimAITE's networking simulation as no supplementary text cells to explain specific output are provided.

## Network Description

<p align="center">
    <a href="./_package_data/uc7/uc7_network_detailed_svg.svg" target="_blank">
        <img src="./_package_data/uc7/uc7_network_detailed_svg.svg" alt="Image">
    </a>  
    
</p>


In [ ]:
!primaite setup

In [ ]:
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.network.firewall import Firewall
from primaite.game.game import PrimaiteGame
from primaite.simulator.system.services.dns.dns_client import DNSClient
from primaite.simulator.sim_container import Simulation
import yaml
from pprint import pprint
from primaite.config.load import load, _EXAMPLE_CFG

In [ ]:
with open(file=_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_logs'] = True # Save agent logs
env = PrimaiteGymEnv(env_config=cfg)

In [ ]:
uc7_network = env.game.simulation.network
uc7_network.show()

## UC7 Network | Home Office Subnet

In [ ]:
home_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-1")
home_pub_pc_2: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-2")
home_pub_pc_srv: Server = env.game.simulation.network.get_node_by_hostname("HOME-PUB-SRV")
home_pub_rt_dr: Router = env.game.simulation.network.get_node_by_hostname("HOME-PUB-RT-DR")

In [ ]:
# HOME PC 1 --> HOME PC 2
home_pub_pc_1.ping(home_pub_pc_2.network_interface[1].ip_address)

In [ ]:
# HOME PC 2 --> HOME SERVER
home_pub_pc_2.ping(home_pub_pc_srv.network_interface[1].ip_address)

In [ ]:
# HOME SERVER --> HOME ROUTER
home_pub_pc_srv.ping(home_pub_rt_dr.network_interface[1].ip_address)

## UC7 Network | Internet

In [ ]:
isp_pub_rt_br: Router = env.game.simulation.network.get_node_by_hostname("ISP-PUB-RT-BR")
isp_pub_srv_dns: Server = env.game.simulation.network.get_node_by_hostname("ISP-PUB-SRV-DNS")

In [ ]:
home_pub_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# HOME ROUTER --> INTERNET ROUTER
home_pub_rt_dr.ping(isp_pub_rt_br.network_interface[1].ip_address)

In [ ]:
home_pub_rt_dr.route_table.show()

In [ ]:
# HOME ROUTER --> INTERNET ISP
home_pub_rt_dr.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# INTERNET ROUTER --> INTERNET DNS
isp_pub_rt_br.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# HOME ROUTER --> INTERNET DNS
home_pub_rt_dr.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# PC 1 --> INTERNET DNS

home_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Remote Site Subnet

In [ ]:
network: Network = env.game.simulation.network


rem_pub_fw: Firewall = network.get_node_by_hostname(hostname="REM-PUB-FW")
rem_pub_rt_dr: Router = network.get_node_by_hostname(hostname="REM-PUB-RT-DR")
rem_pub_sw_as: Switch = network.get_node_by_hostname(hostname="REM-PUB-SW-AS")
rem_pub_pc_1: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-1")
rem_pub_pc_2: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-2")
rem_pub_srv: Computer = network.get_node_by_hostname(hostname="REM-PUB-SRV")

In [ ]:
# Remote PC 1 --> Remote PC_2

rem_pub_pc_1.ping(rem_pub_pc_2.network_interface[1].ip_address)

In [ ]:
# Remote PC 2 --> Remote Server

rem_pub_pc_2.ping(rem_pub_srv.network_interface[1].ip_address)

In [ ]:
# Remote Server --> Remote Site Router

rem_pub_srv.ping(rem_pub_rt_dr.network_interface[1].ip_address)

In [ ]:
# Remote Site Router --> Remote Pub Firewall (Internal)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[1].ip_address)

In [ ]:
# Remote Site Router --> Remote Pub Firewall (External)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[2].ip_address)

In [ ]:
# Remote Pub Firewall (external) --> Public Internet

rem_pub_fw.ping(isp_pub_rt_br.network_interface[1].ip_address)

In [ ]:
# Remote Site PC 1 --> Public DNS

rem_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Remote Site PC 1 --> Home Office PC 1

rem_pub_pc_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

## UC7 Network | SOME_TECH_DMZ

In [ ]:
st_pub_fw: Firewall = network.get_node_by_hostname(hostname="ST_PUB-FW")
st_dmz_pub_srv_web: Server = network.get_node_by_hostname(hostname="ST_DMZ-PUB-SRV-WEB")

In [ ]:
# Some Tech Firewall --> Some Tech DMZ public web server

st_pub_fw.ping(st_dmz_pub_srv_web.network_interface[1].ip_address)

In [ ]:
# Some Tech Firewall --> Public DNS Server

st_pub_fw.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Some Tech Firewall --> Internet Router (Port 4)

st_pub_fw.ping(isp_pub_rt_br.network_interface[4].ip_address)

## UC7 Network | SOME_TECH_INTRANET

In [ ]:
st_intra_prv_rt_cr: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_dr_1: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-1")
st_intra_prv_rt_dr_2: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-2")

In [ ]:
# Some Tech Intranet Private Router CR --> Some Tech Public Firewall

st_intra_prv_rt_cr.ping(st_pub_fw.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 2 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_2.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some tech intranet router DR 2

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (Internal Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (DMZ Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[2].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Some Tech Firewall (External Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Some Tech DMZ web-server

st_intra_prv_rt_dr_1.ping(target_ip_address=st_dmz_pub_srv_web.network_interface[1].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Public Internet

st_intra_prv_rt_dr_1.ping(isp_pub_rt_br.network_interface[3].ip_address)

In [ ]:
# Some tech intranet router DR 1 --> Public DNS

st_intra_prv_rt_dr_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Home Office PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

In [ ]:
# Some Tech Intranet Router DR 1 --> Remote Site PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=rem_pub_pc_1.network_interface[1].ip_address)

## UC7 Network | Some Tech Head Office

In [ ]:
st_head_office_private_pc_1: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-1")
st_head_office_private_pc_2: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-2")
st_head_office_private_pc_3: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-3")

In [ ]:
# ST Home office PC 1 --> ST Home Office PC 2

st_head_office_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

In [ ]:
# ST Home office PC 3 --> ST Router DR 2

st_head_office_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# ST Home office PC 1 --> Public DNS

st_head_office_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Human Resources

In [ ]:
st_human_resources_private_pc_1: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-1")
st_human_resources_private_pc_2: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-2")
st_human_resources_private_pc_3: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-3")

In [ ]:
# ST Human Resources PC 1 --> ST Human Resources PC 2

st_human_resources_private_pc_1.ping(st_human_resources_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 2 --> # ST Human Resources PC 3

st_human_resources_private_pc_2.ping(st_human_resources_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 1 --> ST Intranet Router DR 2

st_human_resources_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

In [ ]:
# ST Human Resources PC 1 --> Public DNS

st_human_resources_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Data (Database / Database Backup)

In [ ]:
st_data_private_server_database: Server = network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_private_server_storage: Server = network.get_node_by_hostname("ST_DATA-PRV-SRV-STORAGE")

In [ ]:
# ST Data Private Database --> ST Data Private Storage

st_data_private_server_database.ping(st_data_private_server_storage.network_interface[1].ip_address)

In [ ]:
# ST Data Private Storage --> Public DNS

st_data_private_server_storage.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Project A 

In [ ]:
st_project_a_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
st_project_a_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-2")
st_project_a_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-3")

In [ ]:
# ST Private Project A PC 1 --> ST Private Project A PC 2

st_project_a_private_pc_1.ping(st_project_a_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 2 --> ST Private Project A PC 3

st_project_a_private_pc_3.ping(st_project_a_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 3 --> Public DNS

st_project_a_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST_INTRA-PRV-RT-CR

st_project_a_private_pc_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST_INTRA-PRV-RT-DR-2

st_project_a_private_pc_1.ping("192.168.170.2")

## UC7 Network | Some Tech Project B

In [ ]:
st_project_b_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-1")
st_project_b_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-2")
st_project_b_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-3")

In [ ]:
# ST Private Project B PC 1 --> ST Private Project B PC 2

st_project_b_private_pc_1.ping(st_project_b_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project B PC 2 --> ST Private Project B PC 3

st_project_b_private_pc_2.ping(st_project_b_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project B PC 3 --> Public DNS

st_project_b_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

## UC7 Network | Some Tech Project C

In [ ]:
st_project_c_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-1")
st_project_c_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-2")
st_project_c_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-3")

In [ ]:
# ST Private Project C PC 1 --> ST Private Project C PC 2

st_project_c_private_pc_1.ping(st_project_c_private_pc_2.network_interface[1].ip_address)

In [ ]:
# ST Private Project C PC 2 --> ST Private Project C PC 3

st_project_c_private_pc_2.ping(st_project_c_private_pc_3.network_interface[1].ip_address)

In [ ]:
# ST Private Project C PC 3 --> Public DNS

st_project_c_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

In [ ]:
# ST Private Project C PC 1 --> ST Private Project B PC 1

st_project_c_private_pc_1.ping(st_project_b_private_pc_1.network_interface[1].ip_address)

In [ ]:
# ST Private Project B PC 1 --> ST Private Project A PC 1

st_project_b_private_pc_1.ping(st_project_a_private_pc_1.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST Head Office Private PC 1

st_project_a_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

In [ ]:
# ST Private Project A PC 1 --> ST Human Resources Private PC 1

st_project_a_private_pc_1.ping(st_human_resources_private_pc_1.network_interface[1].ip_address)

## UC7 Network | Applications & Services

In [ ]:
# dns-server

isp_pub_srv_dns.software_manager.show()

In [ ]:
# web-browser

st_project_a_web_browser = st_project_a_private_pc_1.software_manager.software["web-browser"]
st_project_a_web_browser.get_webpage()

In [ ]:
# web-server

st_web_server = st_dmz_pub_srv_web.software_manager.software["web-server"]
st_web_server.sys_log.show()

In [ ]:
# database-client

st_database_client = st_project_a_private_pc_1.software_manager.software["database-client"]
st_database_client.connect()

In [ ]:
# database-service

st_database = st_data_private_server_database.software_manager.software["database-service"]
st_database.sys_log.show()

In [ ]:
# ftp-server

st_ftp_server = st_data_private_server_storage.software_manager.software["ftp-server"]

st_ftp_server.sys_log.show()

In [ ]:
# dns-client 

home_pub_pc_1: Computer = network.get_node_by_hostname("HOME-PUB-PC-1")
dns_client: DNSClient = home_pub_pc_1.software_manager.software["dns-client"]

dns_client.check_domain_exists(target_domain="some_tech.com")
dns_client.dns_cache.get("some_tech.com", None)
len(dns_client.dns_cache) == 1


## UC7 Network | Green & Red Agent (TAP001) default behaviour

In [ ]:
for _ in range(100):
    env.step(0)

In [ ]:
def print_agent_actions_except_do_nothing(agent_name):
    print(f"\n{agent_name} actions this episode:")
    for item in env.game.agents[agent_name].history:
        if item.action != "do-nothing":
            node, application = 'unknown', 'unknown'
            if (item.parameters.get('node_name')) is not None:
                node = item.parameters.get('node_name')
            if (item.parameters.get('application_name')) is not None:
                application = item.parameters.get('application_name')
            print(f"Step: {item.timestep}, action: {item.action}, node: {node}, application: {application}, response: {item.response.status}")

print_agent_actions_except_do_nothing("HOME_WORKER-1-DB")

In [ ]:
green_agent = env.game.agents.get("HOME_WORKER-1-DB")
print(f"Green Agent Ref: {green_agent.config.ref} Reward: {green_agent.reward_function.total_reward}")

In [ ]:
print_agent_actions_except_do_nothing("HOME_WORKER-1-WEB")

In [ ]:
green_agent = env.game.agents.get("HOME_WORKER-1-WEB")
print(f"Green Agent Ref: {green_agent.config.ref} Reward: {green_agent.reward_function.total_reward}")

In [ ]:
red_agent = env.game.agents.get("attacker")
red_agent.logger.show()

In [ ]:
# successful TAP001 kill chain

st_project_a_private_pc_1.file_system.show(full=True)

st_data_private_server_database.file_system.show(full=True)